---
layout: post
title: Gradelog Blog
toc: true
comments: true
---

## Overview of Website
Cantella is a site that was created with the intention to give students various resources in which they could use in order to succeed in school. We have flashcards, quizzes, competitive leaderboards, a way to keep track of your grades and study sessions, and more.

## Individual Feature
The individual feature in which I have been tasked to create was a program that allows a user to input their grade for any given subject and additional notes to describe why or for what assignment they received this grade for.


## Input / Output Requests
The grade log feature can be accessed by the following path: **{{site.url}}/cantella_frontend/gradelog**. 

Here you can input data through the frontend which will subsequently be stored into the backend.

### Frontend Demo

**Frontend View:**

<img src="{{site.baseurl}}/images/grade-log-blog/frontend-I.png" width="35%">
<img src="{{site.baseurl}}/images/grade-log-blog/frontend-O.png" width="40%">


**Backend View:**

<img src="{{site.baseurl}}/images/grade-log-blog/backend-O.png" width="70%">
<img src="{{site.baseurl}}/images/grade-log-blog/backend-database-O.png" width="90%">

### Postman
In order to first retrieve raw data from Postman, we first have to send a **POST** request to `localhost:8887/api/authenticate` and input the data:

```
{
  "uid": "niko",
  "password": "123niko"
}
```

Sending this request will output: `Authentication for niko successful`.

This signifies that we are now logged into the site and hence have access to the grade log feature; a feature only accessible when a user is logged in.

Once logged in, one should now send a **GET** request to `localhost:8887/api/gradelog`

In the case of the example above, this would be the output:

```
[
    {
        "date": "2025-01-27 00:06:57",
        "grade": 100.0,
        "id": 1,
        "notes": "Assignment was first Unit Test. It was really easy.",
        "subject": "Math",
        "user_id": 3
    }
]
```



### Database Management
Having the ability to clear, backup, and restore a database are useful, if not essential, for a functioning database to have. The following will demonstrate the usage of **db_init** (for clearing a database), **db_backup** (for backing up a database), and **db_restore** (for restoring a database).

### db_backup (Backup)
To run db_backup, once must go to their backend repository and run the following in their VS Code terminal: `scripts/db_backup.py`. This "command" is not so much a command as it is a path. When we enter the path to the db_backup file, the terminal assumes we want to run the file and so it does. Inputting this filepath outputs the following:

```
Data backed up to the backup directory.
Database backed up to instance/volumes/user_management_bak.db
```

This output is telling us that all the entries in our main database, `user_management.db`, have been stored to the backup database, `user_management_bak.db`. Now when we clear the database with db_init or sending DELETE requests from the frontend or postman, all our data will still be protected and stored in our backup database, regardless of the changes made to the main one.

### db_init (Clear)

To run db_init, once must go to their backend repository and run the following in their VS Code terminal: `scripts/db_init.py`. To reiterate from before, we are essentially typing in the path to db_init as it not invoke the running of the file. Inputting this filepath outputs the following:

```
Warning, you are about to lose all data in the database!
Do you want to continue? (y/n)
```

In our case, we type **y**, as we want to continue and clear all the data in the database.

Doing so outputs:
```
Database backed up to instance/volumes/user_management_bak.db
All tables dropped.
Generating data.
```

This output is telling us that all tables or, more pragmatically speaking, all our data in our databases, have been cleared. Additionally, new data is being generated, assuming you have default data that needs to be automatically added. When it comes to default data, there can be extra output depending if one has added such or not.

Now that we have backed up our data and cleared it from the main database, it's time to restore it back.


### db_restore (Restore)
Same song, different tune. Input `scripts/db_restore.py` to the terminal and you will be outputted with:

```
Data restored to the new database.
```

The data that we backed up to `user_management_bak.db` is now copied back into  `user_management.db`, allowing one to retrieve any data they may have deleted or lost.

## List Requests